In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.losses import BinaryCrossentropy

In [2]:
G_train_df      = pd.read_csv ('data/preprocessed_data/G_train.csv')
T_train_df      = pd.read_csv ('data/preprocessed_data/T_train.csv')
G_T_train_df    = pd.read_csv ('data/preprocessed_data/balanced_G_T_train.csv')

G_cv_df         = pd.read_csv ('data/preprocessed_data/G_cv.csv')
T_cv_df         = pd.read_csv ('data/preprocessed_data/T_cv.csv')
G_T_cv_df        = pd.read_csv ('data/preprocessed_data/G_T_cv.csv')



In [3]:
random_state = 13
frac = 0.1

G_train_df      = G_train_df.sample(frac = frac, random_state=random_state)
T_train_df      = T_train_df.sample(frac = frac, random_state=random_state)
G_T_train_df    = G_T_train_df.sample(frac = frac, random_state=random_state)


In [4]:
G_T_train_df.shape

(12879, 3)

In [5]:
y_train = G_T_train_df.drop(columns = ['group ID','technique ID' ]).values
y_train.dtype

# G_train = sampled_G_train_df.drop(columns = ids)
G_train = G_train_df.drop(columns = 'group ID').values

# T_train = sampled_T_train_df.drop(columns = ids)
T_train = T_train_df.drop(columns = 'technique ID').values

```python
dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_train_tf, 'input_Technique': T_train_tf}, y_train_tf))
```

1. `tf.data.Dataset`: This is a class in TensorFlow that represents a potentially large collection of elements. It's used to create pipelines for data processing and input to machine learning models.

2. `from_tensor_slices`: This is a method of the `tf.data.Dataset` class. It is used to create a dataset by slicing the given tensors along their first dimension (usually representing the number of samples). This is a convenient way to create a dataset where each element corresponds to a pair of inputs and their corresponding targets.

3. `({'input_Group': G_train_tf, 'input_Technique': T_train_tf}, y_train_tf)`: This is a tuple containing two elements. The first element is a dictionary that maps the input tensor names (`'input_Group'` and `'input_Technique'`) to their corresponding tensor data (`G_train_tf` and `T_train_tf`). The second element is the target tensor data `y_train_tf`. ❗The  names used in the from_tensor_slices dictionary should match the names of the input layers in your model. This ensures that the data from the dataset is correctly mapped to the corresponding input layers during training.


So, the overall process of this line of code is to create a dataset where each element is a pair of dictionaries (`{'input_Group': ..., 'input_Technique': ...}`) representing the input tensors and their corresponding target tensor.

When you iterate through this dataset, you'll get pairs like:
```
({'input_Group': G_train_sample, 'input_Technique': T_train_sample}, y_train_sample)
```

Here, `G_train_sample` and `T_train_sample` are individual samples from your `input_Group` and `input_Technique` tensors, and `y_train_sample` is the corresponding target value for that sample. This dataset structure is suitable for training machine learning models where you have multiple input features.

In [6]:
G_train_tf = tf.convert_to_tensor(G_train)
T_train_tf = tf.convert_to_tensor(T_train)
y_train_tf = tf.convert_to_tensor(y_train)
train_dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_train_tf, 'input_Technique': T_train_tf}, y_train_tf))

batch_size = 32


train_dataset = train_dataset.shuffle(buffer_size=len(G_train_tf))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [7]:
y_cv = G_T_cv_df.drop(columns = ['group ID','technique ID' ]).values
y_cv.dtype

# G_cv = sampled_G_cv_df.drop(columns = ids)
G_cv = G_cv_df.drop(columns = 'group ID').values

# T_cv = sampled_T_cv_df.drop(columns = ids)
T_cv = T_cv_df.drop(columns = 'technique ID').values

In [8]:
G_cv_tf = tf.convert_to_tensor(G_cv)
T_cv_tf = tf.convert_to_tensor(T_cv)
y_cv_tf = tf.convert_to_tensor(y_cv)
cv_dataset = tf.data.Dataset.from_tensor_slices(({'input_Group': G_cv_tf, 'input_Technique': T_cv_tf}, y_cv_tf))
cv_dataset = cv_dataset.batch(batch_size)

---

In [9]:
# input shapes config
num_G_features = G_train.shape[1]  # remove Group ID during training
num_T_features = T_train.shape[1]   # remove Movie ID during training

# output
num_outputs = 32

tf.random.set_seed(random_state)

# Group NN
Group_NN = tf.keras.models.Sequential(
    layers=[
    tf.keras.layers.Dense (256, activation = 'relu'),
    tf.keras.layers.Dense (128, activation = 'relu'),
    tf.keras.layers.Dense (128, activation = 'relu'),
    tf.keras.layers.Dense (num_outputs, activation  = 'linear'),
    ], 
    name= "Group_NN")
# input vector for user_NN
input_Group = tf.keras.layers.Input(shape = (num_G_features), name = "input_Group")
vg = Group_NN(input_Group)
# vg = tf.linalg.l2_normalize(vg, axis=1)

# Technique NN
Technique_NN = tf.keras.models.Sequential(
    layers = [
    tf.keras.layers.Dense (256, activation = 'relu'),
    tf.keras.layers.Dense (128, activation = 'relu'),
    tf.keras.layers.Dense (128, activation = 'relu'),
    tf.keras.layers.Dense (num_outputs, activation  = 'linear'),  
    ],
    name = "Technique_NN")
# input vector for Technique_NN
input_Technique = tf.keras.layers.Input (shape= (num_T_features), name = "input_Technique")
vt = Technique_NN (input_Technique)
# vt = tf.linalg.l2_normalize (vt, axis = 1)

output = tf.keras.layers.Dot (axes=1)(inputs= [vg, vt])



In [10]:
tf.random.set_seed(random_state)
opt = keras.optimizers.Adam (learning_rate= 0.05)
model = tf.keras.Model (inputs = [input_Group, input_Technique],
                        outputs = output, name = 'recsysNN_model')
model.compile (optimizer = opt, loss = BinaryCrossentropy (from_logits= True))


## Early stopping

In [11]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss for early stopping
    patience=32,           # Number of epochs with no improvement before stopping
    restore_best_weights=True  # Restore model weights from the epoch with the best validation loss
)

In [12]:
epochs = 100
history = model.fit(train_dataset,
                    validation_data = cv_dataset,
                    epochs = epochs,
                    callbacks = [early_stopping])


Epoch 1/100
403/403 [==============================] - 3s 5ms/step - loss: 1.1060 - val_loss: 0.6375
Epoch 2/100
403/403 [==============================] - 2s 4ms/step - loss: 0.7051 - val_loss: 0.6718
Epoch 3/100
403/403 [==============================] - 2s 4ms/step - loss: 0.7046 - val_loss: 0.6549
Epoch 4/100
403/403 [==============================] - 2s 4ms/step - loss: 0.6940 - val_loss: 0.7066
Epoch 5/100
403/403 [==============================] - 2s 4ms/step - loss: 0.6933 - val_loss: 0.6942
Epoch 6/100
403/403 [==============================] - 2s 5ms/step - loss: 0.6932 - val_loss: 0.6939
Epoch 7/100
403/403 [==============================] - 2s 4ms/step - loss: 0.6932 - val_loss: 0.6949
Epoch 8/100
403/403 [==============================] - 2s 4ms/step - loss: 0.6932 - val_loss: 0.6932
Epoch 9/100
403/403 [==============================] - 2s 4ms/step - loss: 0.6931 - val_loss: 0.6931
Epoch 10/100
403/403 [==============================] - 2s 4ms/step - loss: 0.6931 - val_lo

In [13]:
import csv
training_loss = history.history['loss']
validation_loss = history.history['val_loss']

training_loss_file = 'training_loss.csv'
validation_loss_file = 'validation_loss.csv'

with open(training_loss_file, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(training_loss)

with open(validation_loss_file, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(validation_loss)

In [14]:
print(len(training_loss))
print(len(validation_loss))

33
33


In [15]:
print (training_loss)
print (validation_loss)

[1.1059573888778687, 0.7051196098327637, 0.7046013474464417, 0.6940036416053772, 0.6932709813117981, 0.6931689977645874, 0.6931513547897339, 0.6931578516960144, 0.6931471228599548, 0.6931462287902832, 0.6931464076042175, 0.6931467056274414, 0.6931459903717041, 0.6933404803276062, 0.6933574676513672, 0.6967170238494873, 0.6961445212364197, 0.6953396201133728, 0.6956703066825867, 0.6945726871490479, 0.6939758658409119, 0.6958285570144653, 0.6955490112304688, 0.6956813335418701, 0.6934887170791626, 0.6967905163764954, 0.6989401578903198, 0.6936697363853455, 0.6961933970451355, 0.69358891248703, 0.694071888923645, 0.6959228515625, 0.6948370933532715]
[0.6374624967575073, 0.6718109250068665, 0.6548517346382141, 0.7065610885620117, 0.6941655874252319, 0.693869411945343, 0.6948541402816772, 0.693224310874939, 0.6931468844413757, 0.6931461095809937, 0.6931502819061279, 0.6931461095809937, 0.6931461095809937, 0.746077299118042, 0.7034499645233154, 0.7051395773887634, 0.6906052827835083, 0.65595

In [16]:
# Plot the training and validation loss
# plt.plot(range(1, len(training_loss) + 1), training_loss, label='Training Loss')
# plt.plot(range(1, len(validation_loss) + 1), validation_loss, label='Validation Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.legend()
# plt.show()

In [17]:
# plt.plot(range(1, len(training_loss) + 1), training_loss, label='Training Loss')